# Quick intro to Kernel Memory: install, upload a doc, ask a question

This notebook shows the basic usage of Kernel Memory, via the serverless client.

First of all, install the Kernel Memory dependency.

In [1]:
#r "nuget: Microsoft.KernelMemory.Core, 0.29.240219.2"

using Microsoft.KernelMemory;

Installed Packages Microsoft.KernelMemory.Core, 0.29.240219.2

For our demo, we use also the "dotenv" nuget, to load our secret credentials from a `.env` file.
Make sure you create your `.env` file, with this content:

> ```
> OPENAI_API_KEY=<your OpenAI API key>
> ```

In [2]:
#r "nuget: dotenv.net, 3.1.3"

dotenv.net.DotEnv.Load();
var env = dotenv.net.DotEnv.Read();

Installed Packages dotenv.net, 3.1.3

Now let's create an instance of Kernel Memory client using the `KernelMemoryBuilder` to configure our solution. 

In this demo we use OpenAI to calculate embeddings and generate text, and the default storage settings, with content and embeddings kept in a volatile memory automatically deleted after the execution.

In [3]:
var memory = new KernelMemoryBuilder()
    .WithOpenAIDefaults(env["OPENAI_API_KEY"])
    .Build<MemoryServerless>();

info: Microsoft.KernelMemory.Handlers.TextExtractionHandler[0]
      Handler 'extract' ready
info: Microsoft.KernelMemory.Handlers.TextPartitioningHandler[0]
      Handler 'partition' ready
info: Microsoft.KernelMemory.Handlers.SummarizationHandler[0]
      Handler 'summarize' ready
info: Microsoft.KernelMemory.Handlers.GenerateEmbeddingsHandler[0]
      Handler 'gen_embeddings' ready, 1 embedding generators
info: Microsoft.KernelMemory.Handlers.SaveRecordsHandler[0]
      Handler save_records ready, 1 vector storages
info: Microsoft.KernelMemory.Handlers.DeleteDocumentHandler[0]
      Handler 'private_delete_document' ready
info: Microsoft.KernelMemory.Handlers.DeleteIndexHandler[0]
      Handler 'private_delete_index' ready
info: Microsoft.KernelMemory.Handlers.DeleteGeneratedFilesHandler[0]
      Handler 'delete_generated_files' ready


At this point we have the memory client ready, we can import data and ask questions!

Let's import a PDF file with some details about Kernel Memory:

In [4]:
await memory.ImportDocumentAsync("sample-KM-Readme.pdf", documentId: "doc001");

Now, let's ask a question about Kernel Memory:

In [5]:
var question = "What's Kernel Memory?";

var answer = await memory.AskAsync(question);

Console.WriteLine($"Question: {question}\n\nAnswer: {answer.Result}");

Question: What's Kernel Memory?

Answer: Kernel Memory is a multi-modal AI service that specializes in efficient indexing of datasets through custom continuous data hybrid pipelines. It supports features such as Retrieval Augmented Generation (RAG), synthetic memory, prompt engineering, and custom semantic memory processing. It includes a GPT Plugin, web clients, a .NET library for embedded applications, and can be deployed as a Docker container. Kernel Memory enables natural language querying for obtaining answers from indexed data, complete with citations and links to the original sources. It is designed for seamless integration with popular AI platforms and offers support for various data formats and backends. It also provides features like security filters, long-running ingestion, custom tokenization, document storage, OCR via Azure Document Intelligence, and more. Kernel Memory can be used as a service from any language, tool, or platform, including browser extensions, chatbots, a

And here show the sources used to generate the answer above, what we call "citations":

In [6]:
Console.WriteLine("Sources:\n");

foreach (var x in answer.RelevantSources)
{
    Console.WriteLine($"  - {x.SourceName}  - {x.Link} [{x.Partitions.First().LastUpdate:D}]");
}

Sources:

  - sample-KM-Readme.pdf  - default/doc001/3e0c889e762b413594085c715593bbc7 [Tuesday, February 27, 2024]
